# Create Materialized View with Delta

In [20]:
import pyspark
from delta import *
import pyspark.sql.functions as F

In [21]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

In [22]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [23]:
spark.sql("drop table if exists `my_cool_table`")

DataFrame[]

In [24]:
columns = ["language", "num_speakers"]
data = [("English", "1.5"), ("Mandarin", "1.1"), ("Hindi", "0.6")]
rdd = spark.sparkContext.parallelize(data)
df = rdd.toDF(columns)

In [25]:
df.write.format("delta").saveAsTable("default.my_cool_table")

In [26]:
df = spark.table("my_cool_table")

## Create a regular view

In [27]:
df.filter(F.col("num_speakers") > 1.0).createOrReplaceTempView("my_regular_view")

In [28]:
spark.sql("select * from my_regular_view").show()

+--------+------------+
|language|num_speakers|
+--------+------------+
|Mandarin|         1.1|
| English|         1.5|
+--------+------------+



## Create a materialized view

In [30]:
df.filter(F.col("num_speakers") > 1.0).write.format("delta").saveAsTable("default.my_persisted_view")

In [32]:
df = spark.table("my_persisted_view")

In [33]:
df.createOrReplaceTempView("my_materialized_view")

In [34]:
spark.sql("select * from my_materialized_view").show()

+--------+------------+
|language|num_speakers|
+--------+------------+
|Mandarin|         1.1|
| English|         1.5|
+--------+------------+



## Cleanup

In [35]:
%rm -rf spark-warehouse